In [2]:
%load_ext autoreload
%autoreload 1
%aimport transformations

transformations loaded, version: 0.1


In [50]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 999
pd.options.display.max_rows = 113

import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.preprocessing import StandardScaler, PowerTransformer

from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

RS = 35577 # global random state seed
data_path = './data-raw/'

%aimport transformations
%aimport f

In [4]:
df_data = pd.read_csv(data_path + 'bearing_signals.csv', 
                      usecols=['experiment_id', 'bearing_1_id', 'bearing_2_id', 'timestamp', 'a1_x', 'a1_y', 'a1_z', 'a2_x', 'a2_y', 'a2_z', 'rpm', 'w']) 
                      # skippin hz col as it can be clauclatd from rpm/60

(10265700, 12)

In [42]:
df_lbl = pd.read_csv(data_path + 'bearing_classes.csv', delimiter=';', index_col='bearing_id')
df_lbl

,status
bearing_id,
0,1
1,0
2,0
3,0
4,0
...,...
108,1
109,1
110,1


In [45]:
df_data = pd.merge(df_data, df_lbl, left_on='bearing_2_id', right_on='bearing_id')

In [46]:
df_data.shape
df_data = transformations.drop_rpm_outliers(df_data)
df_data = transformations.drop_w_outliers(df_data)
df_data.shape

(10260610, 13)

(10260610, 13)

In [47]:
def q1(arr):
    return np.quantile(arr, 0.25)
def q3(arr):
    return np.quantile(arr, 0.75)

def iqr(arr):
    return q3(arr)-q1(arr)

def kurtosis(series):
    return series.kurtosis()

In [112]:
df_data.columns

Index(['experiment_id', 'bearing_1_id', 'bearing_2_id', 'timestamp', 'a1_x',
       'a1_y', 'a1_z', 'a2_x', 'a2_y', 'a2_z', 'rpm', 'w', 'status'],
      dtype='object')

In [115]:
r = df_data.groupby('experiment_id')[['a1_x','a1_y', 'a1_z', 'a2_x', 'a2_y', 'a2_z', 'rpm', 'w']].agg([np.min, np.max, np.sum, np.median, np.mean, np.std, np.var, q1, q3, iqr, kurtosis])

In [116]:
r.columns = [' '.join(col).strip() for col in r.columns.values]

In [117]:
r

,a1_x amin,a1_x amax,a1_x sum,a1_x median,a1_x mean,a1_x std,a1_x var,a1_x q1,a1_x q3,a1_x iqr,a1_x kurtosis,a1_y amin,a1_y amax,a1_y sum,a1_y median,a1_y mean,a1_y std,a1_y var,a1_y q1,a1_y q3,a1_y iqr,a1_y kurtosis,a1_z amin,a1_z amax,a1_z sum,a1_z median,a1_z mean,a1_z std,a1_z var,a1_z q1,a1_z q3,a1_z iqr,a1_z kurtosis,a2_x amin,a2_x amax,a2_x sum,a2_x median,a2_x mean,a2_x std,a2_x var,a2_x q1,a2_x q3,a2_x iqr,a2_x kurtosis,a2_y amin,a2_y amax,a2_y sum,a2_y median,a2_y mean,a2_y std,a2_y var,a2_y q1,a2_y q3,a2_y iqr,a2_y kurtosis,a2_z amin,a2_z amax,a2_z sum,a2_z median,a2_z mean,a2_z std,a2_z var,a2_z q1,a2_z q3,a2_z iqr,a2_z kurtosis,rpm amin,rpm amax,rpm sum,rpm median,rpm mean,rpm std,rpm var,rpm q1,rpm q3,rpm iqr,rpm kurtosis,w amin,w amax,w sum,w median,w mean,w std,w var,w q1,w q3,w iqr,w kurtosis
experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-9.410165,7.327991,-15077.931386,-0.079124,-0.176971,1.748528,3.057351,-1.137283,0.786643,1.923926,0.803091,-4.298115,5.638507,-148.251562,0.055072,-0.001740,1.074788,1.155169,-0.607370,0.622879,1.230248,0.585030,-8.657115,9.204820,-3012.779247,-0.014243,-0.035361,1.689479,2.854339,-0.878530,0.946076,1.824606,1.416193,-10.004077,8.687975,9763.105715,0.191588,0.114590,2.450624,6.005556,-1.602094,1.796461,3.398555,-0.207130,-8.647843,9.228967,1987.646165,0.002226,0.023329,2.243094,5.031472,-1.439452,1.540016,2.979468,0.227841,-10.207810,11.813929,21360.516464,0.255007,0.250710,1.995330,3.981340,-0.940743,1.450758,2.391501,0.887165,0.000000,1512.605042,9.222250e+07,1374.045802,1082.423656,495.807713,245825.288216,649.819495,1487.603306,837.783811,-0.679615,0.000001,1.679266,35780.683685,0.462081,0.419961,0.323245,0.104487,0.074330,0.661089,0.586759,-0.905792
2,-7.582435,7.424187,-15193.011635,-0.175320,-0.174032,1.560411,2.434883,-0.944891,0.690446,1.635337,0.956034,-4.676653,4.881431,-666.651997,0.055072,-0.007636,0.944751,0.892555,-0.512735,0.528244,1.040979,0.819961,-8.080924,8.340533,-4657.754718,-0.014243,-0.053353,1.547722,2.395442,-0.878530,0.850044,1.728574,0.892300,-9.437651,8.310358,-15625.438165,-0.091625,-0.178986,2.192104,4.805318,-1.696498,1.324439,3.020938,-0.097596,-7.975060,8.748408,2176.952332,0.098338,0.024936,2.084081,4.343395,-1.343340,1.347793,2.691133,0.364910,-11.104623,8.924198,16941.539082,0.155361,0.194061,1.683472,2.834079,-0.741452,1.052174,1.793626,1.315945,0.000000,1512.605042,9.384590e+07,1353.383459,1074.981634,484.529563,234768.897784,573.248408,1487.603306,914.354898,-1.114918,0.000001,1.907451,43751.252875,0.592651,0.501160,0.376274,0.141582,0.084478,0.802710,0.718232,-1.265826
3,-8.352005,7.231794,-13544.978262,-0.175320,-0.172328,1.640117,2.689985,-1.137283,0.786643,1.923926,0.708640,-4.108846,5.449238,-184.195163,0.055072,-0.002343,1.015238,1.030709,-0.607370,0.622879,1.230248,0.555302,-8.369019,8.052437,-3135.997934,-0.014243,-0.039898,1.592576,2.536300,-0.878530,0.946076,1.824606,0.715607,-12.080972,9.632018,-9576.775177,-0.091625,-0.121842,2.429144,5.900740,-1.790903,1.607652,3.398555,-0.207138,-8.455619,10.190086,2841.323417,0.194450,0.036149,2.294527,5.264853,-1.343340,1.540016,2.883356,0.340638,-11.204268,7.927740,14453.035566,0.155361,0.183881,1.755556,3.081975,-0.841097,1.251466,2.092563,0.833997,0.000000,1512.605042,9.038427e+07,1428.571429,1149.927038,445.825447,198760.329198,853.080569,1487.603306,634.522737,-0.052822,0.000001,2.132678,40325.789953,0.581609,0.513051,0.344483,0.118669,0.147707,0.770754,0.623048,-1.005156
4,-6.524276,5.981243,-12121.183112,-0.079124,-0.149644,1.443702,2.084276,-0.944891,0.690446,1.635337,0.634363,-4.676653,4.881431,-44.452189,0.055072,-0.000549,0.952230,0.906742,-0.512735,0.528244,1.040979,0.490713,-7.024573,8.148469,-3051.966469,-0.014243,-0.037679,1.536901,2.362064,-0.878530,0.850044,1.728574,1.026222,-10.004077,7.649527,-9685.467726,-0.091625,-0.119574,2.298337,5.282353,-1.790903,1.513248,3.304151,-0.386285,-7.782836,7.883401,2497.876525,0

In [132]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler, PowerTransformer

In [149]:
r['cluster'] = KMeans(n_clusters=4, random_state=RS).fit_predict(StandardScaler().fit_transform(r))
r['cluster'].value_counts()

3    41
2    40
0    28
1     3
Name: cluster, dtype: int64

In [150]:
res = pd.merge(r, df_data.groupby('experiment_id')['status'].min(), on='experiment_id')

In [151]:
(res['cluster'] == res['status']).sum()

26

In [152]:
res.loc[res['cluster'] == 2, ['status', 'cluster']].tail(20)

,status,cluster
experiment_id,,
89,0,2
90,0,2
91,0,2
92,0,2
93,0,2
94,0,2
95,0,2
96,0,2
97,0,2


In [153]:
res.loc[:, ['status', 'cluster']].tail(20)

,status,cluster
experiment_id,,
93,0,2
94,0,2
95,0,2
96,0,2
97,0,2
98,0,2
99,0,2
100,0,0
101,1,0
